# Objetivo


> **Programa destinado a contar o número de tokens presentes no corpus adquirido da coleta feita do Repositório Institucional da UFSC.**



# Preparação do ambiente

In [6]:
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')
import os
import msgpack
!pip install ferramentas-basicas-pln -U
from ferramentas_basicas_pln import formatarTexto,removerCaracteresEspeciais,removerCaracteresEstranhos,removerEspacosEmBrancoExtras,transformarTextoSubstituindoCaracteres,coletarTextoDeArquivoTxt
from ferramentas_basicas_pln import STRING_CARACTERES_ESPECIAIS_PADRAO

with open(os.path.join(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings','Lista de stopwords para pré-processamento - WOKE UFSC.txt'),'r',encoding='utf-8') as f:
  stopwords = f.read()
lista_stopwords = sorted(set([palavra.strip() for palavra in stopwords.split('\n') if not (palavra.startswith('#') or palavra.startswith('-'))]),key=len)
# len(lista_stopwords)

Mounted at /content/drive


# Funções

In [8]:
def salvarNumeroDeTokensDoTrabalho(nome_variavel : str,
                                   variavel_em_questao,
                                   pasta_do_trabalho : str = '') -> tuple[bool,str]:
  try:
      if not os.path.exists(pasta_do_trabalho):
          os.makedirs(pasta_do_trabalho)

      variable_bytes = msgpack.packb(variavel_em_questao)

      if not nome_variavel.endswith('.msgpack'):
          nome_variavel += '.msgpack'

      with open(os.path.join(pasta_do_trabalho,nome_variavel),'wb') as f:
          f.write(variable_bytes)
          f.close()
          return True, f'Variável {nome_variavel} salva com sucesso no formato .msgpack'

  except Exception as e:
      error_message = f'{e.__class__.__name__}: {str(e)}'
      return False, error_message

def obterNumeroDeTokens(texto : str, silencio : bool = True) -> int:
  lista_tokens_crus = [palavra for palavra in padronizacaoTextual(texto).split()]
  if not silencio:
    print('Lista tokens crus:',lista_tokens_crus)
  qtd_tokens_crus = len(lista_tokens_crus)
  qtd_vocab_cru = len(set(lista_tokens_crus))
  if not silencio:
    print('Lista vocab cru:',set(lista_tokens_crus))

  lista_tokens_sem_stopwords = [palavra for palavra in lista_tokens_crus if palavra not in lista_stopwords]
  if not silencio:
    print('Lista tokens sem stopwords:',lista_tokens_sem_stopwords)
  qtd_tokens_sem_stopwords = len(lista_tokens_sem_stopwords)
  qtd_vocab_sem_stopwords = len(set(lista_tokens_sem_stopwords))
  if not silencio:
    print('Lista vocab sem stopwords:',set(lista_tokens_sem_stopwords))
  return {'qtd_tokens_crus':qtd_tokens_crus,'qtd_vocab_cru':qtd_vocab_cru,'qtd_tokens_sem_stopwords':qtd_tokens_sem_stopwords,'qtd_vocab_sem_stopwords':qtd_vocab_sem_stopwords}

def padronizacaoTextual(texto : str):
  return formatarTexto(texto=texto,
                       padronizar_links=True,padrao_link='link_url_mask',
                       padronizar_dinheiros=True,padrao_dinheiro='R$',
                       padronizar_emails=True,padrao_email='e_mail_mask')

def listagemDeTrabalhos(lista_de_trabalhos : list,
                        n_programa : int,
                        numero_max_programas : int = 12) -> list:
  dic_programas_trabalhos = {'Prog 1':[],'Prog 2':[],'Prog 3':[],'Prog 4':[],'Prog 5':[],
                             'Prog 6':[],'Prog 7':[],'Prog 8':[],'Prog 9':[],'Prog 10':[],
                             'Prog 11':[],'Prog 12':[],'Prog 13':[],'Prog 14':[],'Prog 15':[]}
  for i in range(len(lista_de_trabalhos)):
    if (i+1) <= numero_max_programas:
      num = i + 1
      dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
    else:
      if (i+1)%numero_max_programas != 0:
        num = (i+1)%numero_max_programas
        dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
      else:
        num = numero_max_programas
        dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
  return dic_programas_trabalhos[f'Prog {n_programa}']

def atualizarConsole(nova_string : str) -> None:
  output.clear()
  print(nova_string)

def openMsgPackFile(full_filepath : str,
                    encoding_type : str = None):
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded

# Execução

In [52]:
caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

caminho_colecoes_word_embeddings = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_tokenizados/Colecoes'

lista_de_colecoes_extracao = [os.path.join(caminho_colecoes_extracao_textos,colecao) for colecao in os.listdir(caminho_colecoes_extracao_textos) if ('.' not in colecao)]

lista_nomes_colecoes_extracao = [os.path.basename(colecao) for colecao in lista_de_colecoes_extracao]

dic_colecoes_extracao = {chave: valor for chave, valor in zip(lista_nomes_colecoes_extracao, lista_de_colecoes_extracao)}

n_programa = 1

string_console = f'Programa {n_programa}\n\n'

for nome_colecao in sorted(dic_colecoes_extracao.keys()):

  print('\n'+'-'*100+'\n'+f'Coleção: {nome_colecao}\n'+'-'*100+'\n')
  colecao = dic_colecoes_extracao[nome_colecao]
  lista_de_anos_colecao_extracao = sorted([os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()])

  string_console += f'{nome_colecao}: OK\n'

  for ano in lista_de_anos_colecao_extracao:

    print(f'Ano: {os.path.basename(ano)}\n')

    lista_trabalhos_ano_colecao_extracao = sorted([os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')])

    if len(lista_trabalhos_ano_colecao_extracao) >= n_programa:

      lista_de_trabalhos_para_este_programa = listagemDeTrabalhos(lista_de_trabalhos=lista_trabalhos_ano_colecao_extracao,n_programa=n_programa)

      for trabalho in lista_de_trabalhos_para_este_programa:
        print(f'{os.path.basename(trabalho)}: ')
        lista_arquivos_trabalho_ano_colecao_extracao = [os.path.join(trabalho,arquivo) for arquivo in os.listdir(trabalho) if arquivo in ['notas_de_rodape.txt','texto_principal.txt']]
        txt_total = ''
        for arquivo in lista_arquivos_trabalho_ano_colecao_extracao:
          try:
            txt_total += coletarTextoDeArquivoTxt(caminho_arquivo=arquivo,tipo_de_encoding='utf-8')+'.  '
          except Exception as e:
            print(f'\n! Problema ao abrir arquivo de texto: {e.__class__.__name__}: {str(e)}.\n{trabalho},{ano},{nome_colecao}.\n')
        if txt_total.strip() != '':
          dic_numero_tokens = obterNumeroDeTokens(txt_total)
          caminho_pasta_salvar_pre_processamento = os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho))

          print(salvarNumeroDeTokensDoTrabalho(nome_variavel='dic_numero_tokens',variavel_em_questao=dic_numero_tokens,pasta_do_trabalho=caminho_pasta_salvar_pre_processamento))
    string_console += f'Ano: {os.path.basename(ano)}: OK\n\n'
  atualizarConsole(string_console)

Programa 1

Administracao: OK
Ano: 2003: OK

Ano: 2004: OK

Ano: 2005: OK

Ano: 2006: OK

Ano: 2007: OK

Ano: 2008: OK

Ano: 2009: OK

Ano: 2010: OK

Ano: 2011: OK

Ano: 2012: OK

Ano: 2013: OK

Ano: 2014: OK

Ano: 2015: OK

Ano: 2016: OK

Ano: 2017: OK

Ano: 2018: OK

Ano: 2019: OK

Ano: 2020: OK

Ano: 2021: OK

Ano: 2022: OK

Ano: 2023: OK

Administracao_Universitaria_Mestrado_Profissional: OK
Ano: 2012: OK

Ano: 2013: OK

Ano: 2014: OK

Ano: 2015: OK

Ano: 2016: OK

Ano: 2017: OK

Ano: 2018: OK

Ano: 2019: OK

Ano: 2020: OK

Ano: 2021: OK

Ano: 2022: OK

Ano: 2023: OK

Agroecossistemas: OK
Ano: 2003: OK

Ano: 2004: OK

Ano: 2005: OK

Ano: 2006: OK

Ano: 2007: OK

Ano: 2008: OK

Ano: 2009: OK

Ano: 2010: OK

Ano: 2011: OK

Ano: 2012: OK

Ano: 2013: OK

Ano: 2014: OK

Ano: 2015: OK

Ano: 2016: OK

Ano: 2017: OK

Ano: 2018: OK

Ano: 2019: OK

Ano: 2020: OK

Ano: 2021: OK

Ano: 2022: OK

Ano: 2023: OK

Agroecossistemas_Mestrado_Profissional: OK
Ano: 2012: OK

Ano: 2013: OK

Ano: 2014: O

# Realizando validação do processo (executar somente quando todos os programas finalizarem)

Caso seja encontrado incongruência entre o número total de arquivos de texto "texto_principal.txt" e entre o número de arquivos de contagem de tokens "dic_numero_tokens.msgpack" para uma dada mesma coleção, será notificado com uma mensagem no console da seguinte forma:

"!!! *nome_coleção* qtd_txt *quantidade de arquivos texto_principal na coleção* qtd_tok *quantidade de arquivos dic_numeros_tokens na coleção*"

In [56]:
caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

caminho_colecoes_word_embeddings = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_tokenizados/Colecoes'

lista_de_colecoes_extracao = [os.path.join(caminho_colecoes_extracao_textos,colecao) for colecao in os.listdir(caminho_colecoes_extracao_textos) if ('.' not in colecao)]

lista_nomes_colecoes_extracao = [os.path.basename(colecao) for colecao in lista_de_colecoes_extracao]

dic_colecoes_extracao = {chave: valor for chave, valor in zip(lista_nomes_colecoes_extracao, lista_de_colecoes_extracao)}


for nome_colecao in sorted(dic_colecoes_extracao.keys()):

  print('\n'+'-'*100+'\n'+f'Coleção: {nome_colecao}\n'+'-'*100+'\n')
  colecao = dic_colecoes_extracao[nome_colecao]
  lista_de_anos_colecao_extracao = sorted([os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()])

  qtd_colecao_txt = 0
  qtd_colecao_tok = 0

  for ano in lista_de_anos_colecao_extracao:

    lista_trabalhos_ano_colecao_extracao = sorted([os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')])

    for trabalho in lista_trabalhos_ano_colecao_extracao:
      # print(f'{os.path.basename(trabalho)}: ')
      qtd_colecao_txt += len([os.path.join(trabalho,arquivo) for arquivo in os.listdir(trabalho) if arquivo == 'texto_principal.txt'])
      qtd_colecao_tok += len([os.path.join(trabalho,arquivo) for arquivo in os.listdir(os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho))) if arquivo == 'dic_numero_tokens.msgpack'])

  if qtd_colecao_txt != qtd_colecao_tok:
    print('\n!!!',nome_colecao,'qtd_txt',qtd_colecao_txt,'qtd_tok',qtd_colecao_tok,'\n')


----------------------------------------------------------------------------------------------------
Coleção: Administracao
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Coleção: Administracao_Universitaria_Mestrado_Profissional
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Coleção: Agroecossistemas
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Coleção: Agroecossistemas_Mestrado_Profissional
----------------------------------------------------------------------------------------------------


---------------------------

# Realizando contagem de tokens no corpus inteiro

In [64]:
caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

caminho_colecoes_word_embeddings = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_tokenizados/Colecoes'

lista_de_colecoes_word_embeddings = [os.path.join(caminho_colecoes_word_embeddings,colecao) for colecao in os.listdir(caminho_colecoes_word_embeddings) if ('.' not in colecao)]

lista_nomes_colecoes_word_embeddings = [os.path.basename(colecao) for colecao in lista_de_colecoes_extracao]

dic_colecoes_word_embeddings = {chave: valor for chave, valor in zip(lista_nomes_colecoes_word_embeddings, lista_de_colecoes_word_embeddings)}


qtd_tokens_crus_total = 0
qtd_tokens_sem_stopwords_total = 0

for contagem,nome_colecao in enumerate(sorted(dic_colecoes_word_embeddings.keys())):
  # print('Coleções', contagem+1,'de',len(dic_colecoes_word_embeddings.keys()))

  colecao = dic_colecoes_word_embeddings[nome_colecao]
  lista_de_anos_colecao_extracao = sorted([os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()])

  for ano in lista_de_anos_colecao_extracao:

    lista_trabalhos_ano_colecao_extracao = sorted([os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')])

    for trabalho in lista_trabalhos_ano_colecao_extracao:
      dic_numero_tokens_trabalho = openMsgPackFile(os.path.join(trabalho,'dic_numero_tokens.msgpack'))
      qtd_tokens_crus_total += dic_numero_tokens_trabalho['qtd_tokens_crus']
      qtd_tokens_sem_stopwords_total += dic_numero_tokens_trabalho['qtd_tokens_sem_stopwords']

  atualizarConsole(f"Coleções {contagem+1} de {len(dic_colecoes_word_embeddings.keys())}\n\nNúmero de tokens até o momento:\n\nqtd_tokens_total = {'{0:,}'.format(qtd_tokens_crus_total).replace(',','.')}\nqtd_tokens_sem_stopwords_total = {'{0:,}'.format(qtd_tokens_sem_stopwords_total).replace(',','.')}")

Coleções 112 de 112

Número de tokens até o momento:

qtd_tokens_total = 1.220.839.374
qtd_tokens_sem_stopwords_total = 826.526.579


# Teste com texto manual para checar processo de contagem de tokens utilizado

In [11]:
texto_teste = 'Esse é um texto de teste e sei que é pequeno, mas já garante uma contagem fiel! Vou botar # caractér especial.'

dic_numero_tokens = obterNumeroDeTokens(texto_teste,silencio=False)
print('\n--- Resultado final ---\n')
for chave in dic_numero_tokens:
  print(chave,':',dic_numero_tokens[chave])

Lista tokens crus: ['esse', 'é', 'um', 'texto', 'de', 'teste', 'e', 'sei', 'que', 'é', 'pequeno', 'mas', 'já', 'garante', 'uma', 'contagem', 'fiel', 'vou', 'botar', 'caractér', 'especial']
Lista vocab cru: {'sei', 'fiel', 'vou', 'especial', 'é', 'caractér', 'garante', 'botar', 'texto', 'um', 'mas', 'contagem', 'esse', 'uma', 'e', 'já', 'pequeno', 'teste', 'que', 'de'}
Lista tokens sem stopwords: ['é', 'um', 'texto', 'teste', 'sei', 'é', 'pequeno', 'garante', 'uma', 'contagem', 'fiel', 'vou', 'botar', 'caractér', 'especial']
Lista vocab sem stopwords: {'texto', 'teste', 'um', 'especial', 'sei', 'contagem', 'fiel', 'é', 'uma', 'caractér', 'garante', 'pequeno', 'botar', 'vou'}

--- Resultado final ---

qtd_tokens_crus : 21
qtd_vocab_cru : 20
qtd_tokens_sem_stopwords : 15
qtd_vocab_sem_stopwords : 14
